In [1]:
import pyodbc
import pandas as pd
from dotenv import load_dotenv
import os
import datetime
import calendar
from datetime import date
import holidays
import numpy as np

In [2]:
load_dotenv("keys.env",override=True)

True

In [3]:
driver = os.environ['DRIVER']
server = os.environ['SERVER']
database = os.environ['DATABASE']

In [4]:
### funcoes 

def conexao(driver, server, database):
    
    cnxn_str = (f"Driver={str(driver)};"
                f"Server={str(server)};"
                f"Database={str(database)};"
                "Trusted_Connection=yes;")
    cnxn = pyodbc.connect(cnxn_str)
    return cnxn


def truncate_table(driver, server, database, tabela_destino):
    
    con = conexao(driver, server, database)
    
    truncate = f"""TRUNCATE TABLE {tabela_destino} """
    
    cursor = con.cursor()
    cursor.execute(truncate)
    
    con.commit()
    cursor.close()
    
    return

### ETL - DIM DATA

In [5]:
### Funcao Carregar Dados fato_grao
def carregar_dados_dim_data(driver, server, database,dados, tabela_destino):
    
    con = conexao(driver, server, database)
  
    query = f'''INSERT INTO {tabela_destino} ''' + '''(''' + ','.join(list(dados)) + ''')''' + ''' values(''' + str('?,' * dados.shape[1])[:-1] + ''')'''
    
    cursor = con.cursor()
    print("Query Feita")
    
    try:
        for index, row in dados.iterrows():
            cursor.execute(query, row.IdData, row.Data, row.Dia, row.Mes, row.Ano, row.Ano_Mes, row.Dia_Semana, row.Nome_Dia_Semana, row.Semana_Ano, row.Semestre, row.Trimestre, row.InFeriado, row.InFds)
        con.commit()
        cursor.close()
    except:
        print('Erro na Inserção dos Dados')
        
    return

In [6]:
print('Driver: ',str(driver))
print('Server: ',str(server))
print('Database: ',str(database))

Driver:  SQL Server
Server:  DESKTOP-4IH8F9C\SQLEXPRESS
Database:  pardiniteste


In [7]:
from datetime import date, timedelta
sdate = date(1900,1,1)   # start date
edate = date(2100,1,1)   # end date

lista_data = pd.date_range(sdate,edate-timedelta(days=1),freq='d')

N = len(lista_data)
id_lista = list(range(1,N+1,1))

d = {'IdData': id_lista,
     'Data': pd.to_datetime(lista_data)}
dim_data = pd.DataFrame(d)

In [8]:
dim_data['Dia'] = list(dim_data['Data'].apply(lambda x: x.day))
dim_data['Mes'] = list(dim_data['Data'].apply(lambda x: x.month))
dim_data['Ano'] = list(dim_data['Data'].apply(lambda x: x.year))
dim_data['Ano_Mes'] = dim_data['Ano']*100 + dim_data['Mes']
dim_data['Dia_Semana'] = list(dim_data['Data'].apply(lambda x: x.weekday()))
dim_data['Nome_Dia_Semana'] = dim_data['Data'].apply(lambda x: calendar.day_name[x.weekday()])
dim_data['Semana_Ano'] = list(dim_data['Data'].apply(lambda x: x.isocalendar().week))
dim_data['Semestre'] = list(np.where(dim_data['Data'].apply(lambda x: x.quarter) < 3,1,2))
dim_data['Trimestre'] = list(dim_data['Data'].apply(lambda x: x.quarter))

In [9]:
feriados = holidays.Brazil()
feriados = feriados['1900-01-01': '2100-01-01']
feriados = pd.DataFrame({'Data': feriados,
              'InFeriado': 1})

feriados['Data'] = feriados['Data'].astype(str)
dim_data['Data'] = dim_data['Data'].astype(str)
dim_data = pd.merge(dim_data, feriados,on = 'Data', how = 'left')
dim_data['InFeriado'] = dim_data['InFeriado'].fillna(0)

In [10]:
dim_data['InFds'] = list(np.where(dim_data['Dia_Semana'] > 4,1,0))

In [11]:
dim_data['Data'] = pd.to_datetime(dim_data['Data'])

In [12]:
dim_data['InFeriado'] = dim_data['InFeriado'].astype(int)

In [13]:
carregar_dados_dim_data(driver, server, database, dim_data, 'dim_data')

Query Feita


In [18]:
#', row.'.join(list(dim_data))